# JSONata Field Mapping Analysis
This notebook analyses a JSONata script and a JSON document, identifying fields in the JSON document that aren't mapped in the JSONata script.

In [1]:

import json
import re
from typing import List, Dict, Any


## Function Definitions

In [2]:

def extract_jsonata_references(jsonata_script: str) -> List[str]:
    """
    Extracts all references within backticks from a JSONata script.

    Args:
    jsonata_script (str): The JSONata script as a string.

    Returns:
    List[str]: A list of references found within backticks.
    """
    # Regular expression to find all occurrences of backtick-enclosed strings
    pattern = r"`([^`]*)`"
    return re.findall(pattern, jsonata_script)


In [3]:

def compare_fields(jsonata_references: List[str], json_data: Dict[str, Any]) -> List[str]:
    """
    Compares JSONata references to the keys in a JSON document.

    Args:
    jsonata_references (List[str]): List of JSONata references.
    json_data (Dict[str, Any]): The JSON document as a dictionary.

    Returns:
    List[str]: A list of JSON keys that are not mapped in the JSONata references.
    """
    unmapped_fields = []

    # Flattening the JSON keys to a single level for comparison
    def flatten_json(y):
        out = {}

        def flatten(x, name=''):
            if type(x) is dict:
                for a in x:
                    flatten(x[a], name + a + '.')
            elif type(x) is list:
                i = 0
                for a in x:
                    flatten(a, name + str(i) + '.')
                    i += 1
            else:
                out[name[:-1]] = x

        flatten(y)
        return out

    flattened_json = flatten_json(json_data)

    # Checking each flattened JSON key against JSONata references
    for key in flattened_json.keys():
        if key not in jsonata_references:
            unmapped_fields.append(key)

    return unmapped_fields


## Execution

In [4]:

# Function to read a file and return its content
def read_file(file_path):
    with open(file_path, 'r') as file:
        return file.read()

# Example file paths (replace with actual file paths)
jsonata_script_path = '../shared/JSONataRegistry/Crew/CrewProductMap.jsonata'
json_document_path = '../shared/DataLibrary/inputs/Crew/Product/JSON/document_1.json'

# Reading the JSONata script and JSON document
jsonata_script = read_file(jsonata_script_path)
json_document = json.loads(read_file(json_document_path))

# Extract references from JSONata script
references = extract_jsonata_references(jsonata_script)

# Compare with JSON document
unmapped_fields = compare_fields(references, json_document)

unmapped_fields


['discontinued']